# Acquire

1st step is to aquire the data from the Codeup db.

In [18]:
#Libraries

# ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Wrangling
import pandas as pd
import numpy as np

# Exploring
import scipy.stats as stats

# Visualizing
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# default pandas decimal number display format
pd.options.display.float_format = '{:20,.2f}'.format

# import acquire
# import summarize
# import prepare

In [19]:
import acquire

In [11]:
import os
import env

In [3]:
# Add how handling nulls and document how we're handling them.
# Handle outliers and document (add to README).

In [ ]:
'''
This function will allow the user to retrieve all tables from the Zillow database from the Codeup DB source. 
It will acquire the data, import it as a dataframe, and then write that dataframe to a .csv file in the local directory.
'''

In [5]:
sql_query = '''
                SELECT * 
                    FROM properties_2017
                    JOIN (select id, logerror, pid, tdate FROM predictions_2017 pred_2017
                    JOIN (SELECT parcelid AS pid, Max(transactiondate) as tdate FROM predictions_2017 GROUP BY parcelid) AS sq1
                    ON (pred_2017.parcelid = sq1.pid AND pred_2017.transactiondate = sq1.tdate)) AS sq2
                    ON (properties_2017.parcelid = sq2.pid)
                    LEFT JOIN airconditioningtype USING (airconditioningtypeid)
                    LEFT JOIN architecturalstyletype USING (architecturalstyletypeid)
                    LEFT JOIN buildingclasstype USING (buildingclasstypeid)
                    LEFT JOIN heatingorsystemtype USING (heatingorsystemtypeid)
                    LEFT JOIN propertylandusetype USING (propertylandusetypeid)
                    LEFT JOIN storytype USING (storytypeid)
                    LEFT JOIN typeconstructiontype USING (typeconstructiontypeid)
                    LEFT JOIN unique_properties USING (parcelid)
                    WHERE latitude IS NOT NULL AND longitude IS NOT NULL;
                '''

In [12]:
# Importing and displaying the zillow dataframe

df = acquire.get_zillow_data()
df.head()

,parcelid,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,id,basementsqft,...,logerror,pid,tdate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,14297519,nan,nan,261.00,nan,nan,nan,nan,1727539,nan,...,0.03,14297519,2017-01-01,None,None,None,None,Single Family Residential,None,None
1,17052889,nan,nan,261.00,nan,nan,nan,nan,1387261,nan,...,0.06,17052889,2017-01-01,None,None,None,None,Single Family Residential,None,None
2,14186244,nan,nan,261.00,nan,nan,nan,nan,11677,nan,...,0.01,14186244,2017-01-01,None,None,None,None,Single Family Residential,None,None
3,12177905,nan,nan,261.00,2.00,nan,nan,nan,2288172,nan,...,-0.10,12177905,2017-01-01,None,None,None,Central,Single Family Residential,None,None
4,10887214,nan,nan,266.00,2.00,nan,nan,1.00,1970746,nan,...,0.01,10887214,2017-01-01,Central,None,None,Central,Condominium,None,None


In [17]:
# Finding the shape of the dataframe to make sure it matches the shape of the data that I built in MySQLPro.

df.shape

(77381, 70)

In [15]:
# Using .info() to find the dtypes of the columns, and to have a baseline count of non-nulls in each column to compare to my nulls when I get into the prep stage.

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77381 entries, 0 to 77380
Data columns (total 70 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   parcelid                      77381 non-null  int64  
 1   typeconstructiontypeid        222 non-null    float64
 2   storytypeid                   50 non-null     float64
 3   propertylandusetypeid         77381 non-null  float64
 4   heatingorsystemtypeid         49440 non-null  float64
 5   buildingclasstypeid           15 non-null     float64
 6   architecturalstyletypeid      206 non-null    float64
 7   airconditioningtypeid         24953 non-null  float64
 8   id                            77381 non-null  int64  
 9   basementsqft                  50 non-null     float64
 10  bathroomcnt                   77381 non-null  float64
 11  bedroomcnt                    77381 non-null  float64
 12  buildingqualitytypeid         49672 non-null  float64
 13  c

In [16]:
df.describe()

,parcelid,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,id,basementsqft,...,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyyear,censustractandblock,id,logerror,pid
count,"77,381.00",222.00,50.00,"77,381.00","49,440.00",15.00,206.00,"24,953.00","77,381.00",50.00,...,"77,269.00","77,380.00","77,381.00","77,379.00","77,376.00","2,886.00","77,137.00","77,381.00","77,381.00","77,381.00"
mean,"13,007,150.51",6.04,7.00,261.83,3.92,3.93,7.39,1.81,"1,495,138.90",679.72,...,"189,314.53","490,134.48","2,016.00","301,095.41","5,995.53",14.09,"60,496,733,646,264.16","38,848.69",0.02,"13,007,150.51"
std,"3,481,345.65",0.56,0.00,5.14,3.59,0.26,2.73,2.97,"860,907.05",689.70,...,"230,087.42","653,443.97",0.00,"492,596.03","7,622.84",2.19,"1,535,241,981,138.57","22,402.43",0.17,"3,481,345.65"
min,"10,711,855.00",4.00,7.00,31.00,1.00,3.00,2.00,1.00,349.00,38.00,...,44.00,"1,000.00","2,016.00",161.00,19.92,3.00,"60,371,011,101,000.00",0.00,-4.66,"10,711,855.00"
25%,"11,538,305.00",6.00,7.00,261.00,2.00,4.00,7.00,1.00,"752,070.00",273.00,...,"84,265.00","207,000.00","2,016.00","85,504.00","2,715.63",14.00,"60,373,109,005,002.00","19,461.00",-0.02,"11,538,305.00"
50%,"12,531,568.00",6.00,7.00,261.00,2.00,4.00,7.00,1.00,"1,497,932.00",515.00,...,"136,499.00","358,975.50","2,016.00","203,372.00","4,450.69",15.00,"60,376,032,003,008.00","38,870.00",0.01,"12,531,568.00"
75%,"14,211,831.00",6.00,7.00,266.00,7.00,4.00,7.00,1.00,"2,240,535.00",796.50,...,"218,787.00","569,001.50","2,016.00","366,796.50","6,927.79",15.00,"60,590,423,251,008.00","58,253.00",0.04,"14,211,831.00"
max,"167,689,317.00",13.00,7.00,275.00,24.00,4.00,21.00,13.00,"2,982,274.00","3,560.00",...,"11,421,790.00","49,061,236.00","2,016.00","48,952,198.00","586,639.30",99.00,"483,030,105,084,015.00","77,613.00",5.26,"167,689,317.00"
